In [2]:
!pip install -q --upgrade tensorflow-graphics tensorflow-datasets
!pip install -q trimesh

     |████████████████████████████████| 4.3 MB 9.4 MB/s 
     |████████████████████████████████| 4.3 MB 39.8 MB/s 
     |████████████████████████████████| 1.1 MB 39.6 MB/s 
     |████████████████████████████████| 646 kB 68.9 MB/s 
     |████████████████████████████████| 281 kB 71.0 MB/s 
     |████████████████████████████████| 98 kB 6.8 MB/s 


In [3]:
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_graphics as tfg

from tensorflow_graphics.datasets.shapenet import Shapenet
from tensorflow_graphics.geometry.representation.mesh import utils as mesh_utils
from tensorflow_graphics.nn.layer.graph_convolution import FeatureSteeredConvolutionKerasLayer
from tensorflow_graphics.notebooks import mesh_segmentation_dataio as mseg_dio

import trimesh

from google.colab import auth
auth.authenticate_user()

In [4]:
(ds_train, ds_val, ds_test), info = Shapenet.load(split=['train', 'validation', 'test'], data_dir='gs://shapenet-dataset-eu/prepared/', with_info=True)

In [5]:
ds_test

<PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'model_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'trimesh': {'faces': TensorSpec(shape=(None, 3), dtype=tf.uint64, name=None), 'vertices': TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)}}>

In [6]:
info

tfds.core.DatasetInfo(
    name='shapenet',
    full_name='shapenet/shapenet_trimesh/1.0.0',
    description="""
    ShapeNetCore is a densely annotated subset of ShapeNet covering 55 common object
    categories with ~51,300 unique 3D models. Each model in ShapeNetCore is linked
    to an appropriate synset in WordNet (version 3.0).
    
    The synsets will be extracted from the taxonomy.json file in the ShapeNetCore.v2.zip
    archive and the splits from http://shapenet.cs.stanford.edu/shapenet/obj-zip/SHREC16/all.csv
    """,
    config_description="""
    
    ShapeNetCore is a densely annotated subset of ShapeNet covering 55 common object
    categories with ~51,300 unique 3D models. Each model in ShapeNetCore is linked
    to an appropriate synset in WordNet (version 3.0).
    
    The synsets will be extracted from the taxonomy.json file in the ShapeNetCore.v2.zip
    archive and the splits from http://shapenet.cs.stanford.edu/shapenet/obj-zip/SHREC16/all.csv
    
    """,
    

In [8]:
for ex in ds_train.take(1):
  trio = trimesh.Trimesh(ex['trimesh']['vertices'], ex['trimesh']['faces'])
  print(ex['label'])

trio.show()

Output hidden; open in https://colab.research.google.com to view.

## Dataset Preparation

In [7]:
MAX_VERTICES = 1000
BATCH_SIZE = 8

ds_train = ds_train.filter(lambda x: tf.shape(x['trimesh']['vertices'])[0] < MAX_VERTICES)
ds_val = ds_val.filter(lambda x: tf.shape(x['trimesh']['vertices'])[0] < MAX_VERTICES)
ds_test = ds_test.filter(lambda x: tf.shape(x['trimesh']['vertices'])[0] < MAX_VERTICES)

In [8]:
def maximum(x, y):
    return tf.math.maximum(tf.cast(tf.shape(y['trimesh']['vertices'])[0], tf.float32), x)

def minimum(x, y):
    return tf.math.minimum(tf.cast(tf.shape(y['trimesh']['vertices'])[0], tf.float32), x)

max_num_vertices = (ds_train
                    .concatenate(ds_val)
                    .concatenate(ds_test)
                    .reduce(0., maximum).numpy().astype(np.int32))

#min_num_vertices = (ds_train
#                    .concatenate(ds_val)
#                    .concatenate(ds_test)
#                    .reduce(float('inf'), minimum).numpy().astype(np.int32))
print(f"We construct our model for a target size of {max_num_vertices} vertices.")

We construct our model for a target size of 999 vertices.


In [9]:
@tf.function
def unique_edges(faces, directed_edges=True):
    faces_typed = tf.cast(faces, tf.int32)
    edges = tf.concat([faces_typed[:, 0:2], 
                       faces_typed[:, 1:3], 
                       tf.gather(faces_typed, [2, 0], axis=-1)],
                      axis=0)
    if directed_edges:
        edges = tf.concat([edges, tf.reverse(edges, axis=[-1])], axis=0)
    return edges

@tf.function
def calculate_neighbors(ex):
    output = {
        'trimesh': {
            'faces': ex['trimesh']['faces'],
            'vertices': ex['trimesh']['vertices']
        },
        'model_id': ex['model_id'],
        'label': ex['label']
    }
    faces = tf.cast(ex['trimesh']['faces'], tf.int64)
    num_vertices = tf.expand_dims(tf.shape(ex['trimesh']['vertices'])[-2], axis=0)
    edges = tf.expand_dims(unique_edges(ex['trimesh']['faces']), axis=0)
    num_edges = tf.expand_dims(tf.shape(edges)[-2], axis=0)
    weights = tf.cast(tf.ones(tf.shape(edges)[:-1]), tf.float32)
    neighbors = mseg_dio.adjacency_from_edges(edges, weights, num_edges, num_vertices)
    output['trimesh']['neighbors'] = tf.sparse.reshape(neighbors, tf.shape(neighbors)[-2:])
    
    return output

def pad_vertices(input):
    return tf.pad(input,  [[0, max_num_vertices - tf.shape(input)[0]], [0, 0]])

@tf.function
def extract_features(ex):
    #'faces': ex['trimesh']['faces']
    return ({'vertices': pad_vertices(ex['trimesh']['vertices']), 'neighbors': ex['trimesh']['neighbors']}, ex['label'])


data_train = (ds_train
            .map(calculate_neighbors, num_parallel_calls=tf.data.AUTOTUNE)  
            .map(extract_features, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(tf.data.AUTOTUNE))

data_val = (ds_val
            .map(calculate_neighbors, num_parallel_calls=tf.data.AUTOTUNE)  
            .map(extract_features, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(tf.data.AUTOTUNE))

data_test = (ds_test
            .map(calculate_neighbors, num_parallel_calls=tf.data.AUTOTUNE)  
            .map(extract_features, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(tf.data.AUTOTUNE))

## PointNet Example

In [10]:
def conv_bn(x, filters):
    x = tf.keras.layers.Conv1D(filters, kernel_size=1, padding='valid')(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.0)(x)
    return tf.keras.layers.ReLU()(x)


def dense_bn(x, filters):
    x = tf.keras.layers.Dense(filters)(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.0)(x)
    return tf.keras.layers.ReLU()(x)


def create_pointnet_model(conv_filters=[32, 32, 32, 64, 512], conv_filter_factor=1, dense_hidden_units=256, dropout=.1):
    inputs = tf.keras.Input(shape=(max_num_vertices, 3), name='vertices')
    
    x = conv_bn(inputs, int(conv_filters[0]*conv_filter_factor))
    for filters in conv_filters[1:]:
        x = conv_bn(x, int(filters*conv_filter_factor))
        
    x = tf.keras.layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, dense_hidden_units)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = dense_bn(x, dense_hidden_units // 2)
    x = tf.keras.layers.Dropout(dropout)(x)

    outputs = tf.keras.layers.Dense(info.features['label'].num_classes)(x)

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    return model

In [13]:
pointnet_model = create_pointnet_model(conv_filters=[32, 64], conv_filter_factor=1, dense_hidden_units=64, dropout=.1)
pointnet_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vertices (InputLayer)       [(None, 999, 3)]          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 999, 32)           128       
                                                                 
 batch_normalization_5 (Batc  (None, 999, 32)          128       
 hNormalization)                                                 
                                                                 
 re_lu_5 (ReLU)              (None, 999, 32)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 999, 64)           2112      
                                                                 
 batch_normalization_6 (Batc  (None, 999, 64)          256       
 hNormalization)                                           

In [14]:
pointnet_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=['accuracy'])

history = pointnet_model.fit(data_train, 
                    validation_data=data_val, 
                    epochs=5)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['neighbors'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


779/779 [==============================] - 227s 284ms/step - loss: 2.2940 - accuracy: 0.5382 - val_loss: 1.4285 - val_accuracy: 0.6277
Epoch 2/5
779/779 [==============================] - 192s 243ms/step - loss: 1.3795 - accuracy: 0.6364 - val_loss: 1.4325 - val_accuracy: 0.6538
Epoch 3/5
779/779 [==============================] - 207s 262ms/step - loss: 1.2343 - accuracy: 0.6574 - val_loss: 1.2623 - val_accuracy: 0.6745
Epoch 4/5
779/779 [==============================] - 182s 230ms/step - loss: 1.1434 - accuracy: 0.6813 - val_loss: 1.2255 - val_accuracy: 0.6835
Epoch 5/5
779/779 [==============================] - 190s 240ms/step - loss: 1.0846 - accuracy: 0.6898 - val_loss: 1.1954 - val_accuracy: 0.7005


In [ ]:
pointnet_model.evaluate(data_test)

## FeastNet Example

In [ ]:
def create_feastnet_model(features_filters=16, weight_matrices=8, out_channels=32, feast_layers=3, conv_out_filters=128, hidden_units=64):
    vertices = tf.keras.Input(name='vertices', shape=(max_num_vertices, info.features['trimesh']['vertices'].shape[-1]))
    neighbors = tf.keras.Input(name='neighbors', shape=(max_num_vertices, max_num_vertices), sparse=True)

    conv = tf.keras.layers.Conv1D(features_filters, 1, activation=None)(vertices)
    for i in range(feast_layers):
        conv = FeatureSteeredConvolutionKerasLayer(num_weight_matrices=weight_matrices, num_output_channels=out_channels*2**i)([conv, neighbors])
        conv = tf.keras.layers.ReLU()(conv)
    graph_conv_output = tf.keras.layers.Conv1D(conv_out_filters, 1, activation='relu')(conv)
    graph_conv_output = tf.reduce_max(graph_conv_output, axis=1, keepdims=False)
    fc1 = tf.keras.layers.Dense(hidden_units, activation='relu')(graph_conv_output)
    fc2 = tf.keras.layers.Dense(hidden_units // 2, activation='relu')(fc1)
    outputs = tf.keras.layers.Dense(info.features['label'].num_classes, activation=None)(fc2)
    
    return tf.keras.Model(inputs=[vertices, neighbors], outputs=outputs)

In [ ]:
feastnet_model = create_feastnet_model(features_filters=16, weight_matrices=8, out_channels=16, feast_layers=2, conv_out_filters=64, hidden_units=64)
feastnet_model.summary()

In [ ]:
feastnet_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=['accuracy'])

history = feastnet_model.fit(data_train, 
                    validation_data=data_val, 
                    epochs=5)

In [ ]:
feastnet_model.evaluate(data_test)